In [1]:
# 패션 MNIST - 훈련세트, 테스트 세트
from tensorflow import keras
(train_input, train_target), (test_input, test_target) = keras.datasets.fashion_mnist.load_data()

In [2]:
# 모델 생성 함수 - 층 추가 가능
def model_fn(a_layer = None):
    model = keras.Sequential()
    model.add(keras.layers.Input(shape=(28,28))) # 입력층
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(100, activation='relu'))

    if a_layer:
        model.add(a_layer)
    
    model.add(keras.layers.Dense(10, activation='softmax'))

    return model

In [3]:
model = model_fn(keras.layers.Dropout(0.3))
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ flatten (Flatten)                    │ (None, 784)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 100)                 │          78,500 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 100)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 10)                  │           1,010 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 79,510 (310.59 KB)

 Trainable params: 79,510 (310.59 KB)

 Non-trainable params: 0 (0.00 B)

In [12]:
train_scaled = train_input / 255
test_scaled = test_input / 255
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# 검증 세트 정확도가 떨어지기 직전 시점에 모델을 파일로 저장
# 과대적합 직전 시점에 모델을 파일로 저장
checkpoint_cb = keras.callbacks.ModelCheckpoint("best-model.keras", save_best_only=True)

# 과대적합 되기 직전까지만 훈련하고 종료 - 조기종료
#restore_best_weights : 가장 최적의 가중치로 복구
early_stopping_cb = keras.callbacks.EarlyStopping(patience=2, restore_best_weights=True) # 검증 세트와 정확도가 2회 이상 올라가지 않으면 조기 종료

model.fit(train_scaled, train_target, epochs=20, validation_data=(test_scaled, test_target), callbacks=[checkpoint_cb, early_stopping_cb])

Epoch 1/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.8927 - loss: 0.2849 - val_accuracy: 0.8824 - val_loss: 0.3424
Epoch 2/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.8945 - loss: 0.2814 - val_accuracy: 0.8803 - val_loss: 0.3411
Epoch 3/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.8932 - loss: 0.2786 - val_accuracy: 0.8822 - val_loss: 0.3537
Epoch 4/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.8986 - loss: 0.2719 - val_accuracy: 0.8848 - val_loss: 0.3384
Epoch 5/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.8982 - loss: 0.2679 - val_accuracy: 0.8820 - val_loss: 0.3491
Epoch 6/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9001 - loss: 0.2658 - val_accuracy: 0.8838 - val_loss: 0.3454


In [13]:
model = keras.models.load_model("best-model.keras")

In [14]:
model.evaluate(test_scaled, test_target)

313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 807us/step - accuracy: 0.8860 - loss: 0.3308


[0.33839860558509827, 0.8848000168800354]

In [15]:
#조기 종료 실험
early_stopping_cb.stopped_epoch

5